# Load libraries

In [1]:
# load pandas and numpy
import pandas as pd
import numpy as np
import re
import os
import sys

# Read and create table for no-covariates models

In [2]:
models = ['arima', 'nhits', 'tft', 'linreg', 'xgboost', 'transformer']
datasets = ['weinstock', 'dubosson', 'colas', 'iglu', 'hall']
time_steps = 12

# create dataframes for MSE and MAE with columns for each dataset and rows for each model
df_mse = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] + 
                      [dataset + ' OOD' for dataset in datasets], index=models)
df_mae = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_likelihood = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models[1:]) # no likelihood for ARIMA
df_calibration = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models[1:]) # no calibration for ARIMA
for model in models:
    for dataset in datasets:
        filename = f"../output/{model}_{dataset}.txt"
        if not os.path.isfile(filename):
            continue
        with open(filename, 'r') as f:
            for line in f:
                if line.startswith('ID median of (MSE, MAE):'):
                    id_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' ID'] = float(id_mse_mae[0])
                    df_mae.loc[model, dataset + ' ID'] = float(id_mse_mae[1])
                elif line.startswith('OOD median of (MSE, MAE):'):
                    ood_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' OOD'] = float(ood_mse_mae[0])
                    df_mae.loc[model, dataset + ' OOD'] = float(ood_mse_mae[1])
                elif line.startswith('ID likelihoods:'):
                    id_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' ID'] = float(id_likelihoods[0])
                elif line.startswith('OOD likelihoods:'):
                    ood_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' OOD'] = float(ood_likelihoods[0])
                elif line.startswith('ID calibration errors:'):
                    id_calib = re.findall(r'-?\d+\.\d+', line)
                    id_calib = np.mean([float(x) for x in id_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' ID'] = id_calib
                elif line.startswith('OOD calibration errors:'):
                    ood_calib = re.findall(r'-?\d+\.\d+', line)
                    ood_calib = np.mean([float(x) for x in ood_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' OOD'] = ood_calib
# sort columns for all dataframes
df_mse = df_mse.reindex(sorted(df_mse.columns), axis=1)
df_mae = df_mae.reindex(sorted(df_mae.columns), axis=1)
df_likelihood = df_likelihood.reindex(sorted(df_likelihood.columns), axis=1)
df_calibration = df_calibration.reindex(sorted(df_calibration.columns), axis=1)


In [3]:
df_wocov_mse = df_mse
df_wocov_mae = df_mae
df_wocov_likelihood = df_likelihood
df_wocov_calibration = df_calibration

# Read and create table for with-covariates models

In [4]:
models = ['nhits', 'tft', 'linreg', 'xgboost', 'transformer']
datasets = ['weinstock', 'dubosson', 'colas', 'iglu', 'hall']
time_steps = 12

# create dataframes for MSE and MAE with columns for each dataset and rows for each model
df_mse = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] + 
                      [dataset + ' OOD' for dataset in datasets], index=models)
df_mae = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_likelihood = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
df_calibration = pd.DataFrame(columns=[dataset + ' ID' for dataset in datasets] +
                        [dataset + ' OOD' for dataset in datasets], index=models)
for model in models:
    for dataset in datasets:
        filename = f"../output/{model}_covariates_{dataset}.txt"
        if not os.path.isfile(filename):
            continue
        with open(filename, 'r') as f:
            for line in f:
                if line.startswith('ID median of (MSE, MAE):'):
                    id_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' ID'] = float(id_mse_mae[0])
                    df_mae.loc[model, dataset + ' ID'] = float(id_mse_mae[1])
                elif line.startswith('OOD median of (MSE, MAE):'):
                    ood_mse_mae = re.findall(r'\d+\.\d+', line)
                    df_mse.loc[model, dataset + ' OOD'] = float(ood_mse_mae[0])
                    df_mae.loc[model, dataset + ' OOD'] = float(ood_mse_mae[1])
                elif line.startswith('ID likelihoods:'):
                    id_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' ID'] = float(id_likelihoods[0])
                elif line.startswith('OOD likelihoods:'):
                    ood_likelihoods = re.findall(r'-?\d+\.\d+', line)
                    df_likelihood.loc[model, dataset + ' OOD'] = float(ood_likelihoods[0])
                elif line.startswith('ID calibration errors:'):
                    id_calib = re.findall(r'-?\d+\.\d+', line)
                    id_calib = np.mean([float(x) for x in id_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' ID'] = id_calib
                elif line.startswith('OOD calibration errors:'):
                    ood_calib = re.findall(r'-?\d+\.\d+', line)
                    ood_calib = np.mean([float(x) for x in ood_calib[:time_steps]])
                    df_calibration.loc[model, dataset + ' OOD'] = ood_calib
# sort columns for all dataframes
df_mse = df_mse.reindex(sorted(df_mse.columns), axis=1)
df_mae = df_mae.reindex(sorted(df_mae.columns), axis=1)
df_likelihood = df_likelihood.reindex(sorted(df_likelihood.columns), axis=1)
df_calibration = df_calibration.reindex(sorted(df_calibration.columns), axis=1)

In [5]:
df_cov_mse = df_mse
df_cov_mae = df_mae
df_cov_likelihood = df_likelihood
df_cov_calibration = df_calibration

# Table 5: Accuracy

In [6]:
# rename iglu to Broll
df_cov_mse = df_cov_mse.rename(columns={'iglu ID': 'Broll ID', 'iglu OOD': 'Broll OOD'})
df_cov_mae = df_cov_mae.rename(columns={'iglu ID': 'Broll ID', 'iglu OOD': 'Broll OOD'})
df_wocov_mse = df_wocov_mse.rename(columns={'iglu ID': 'Broll ID', 'iglu OOD': 'Broll OOD'})
df_wocov_mae = df_wocov_mae.rename(columns={'iglu ID': 'Broll ID', 'iglu OOD': 'Broll OOD'})
# sort columns for all dataframes
df_cov_mse = df_cov_mse.reindex(sorted(df_cov_mse.columns), axis=1)
df_cov_mae = df_cov_mae.reindex(sorted(df_cov_mae.columns), axis=1)
df_wocov_mse = df_wocov_mse.reindex(sorted(df_wocov_mse.columns), axis=1)
df_wocov_mae = df_wocov_mae.reindex(sorted(df_wocov_mae.columns), axis=1)
# convert mase and mse dataframes to float
df_cov_mse = df_cov_mse.astype(float)
df_cov_mae = df_cov_mae.astype(float)
df_wocov_mse = df_wocov_mse.astype(float)
df_wocov_mae = df_wocov_mae.astype(float)
# take square root of all values in the MSE dataframes
df_cov_mse = np.sqrt(df_cov_mse)
df_wocov_mse = np.sqrt(df_wocov_mse)
# get only columns that end with ID
id_cols = [col for col in df_wocov_mse.columns if col.endswith('ID')]
# get only columns that end with OOD
ood_cols = [col for col in df_wocov_mse.columns if col.endswith('OOD')]

In [7]:
# calculate differene between with and without covs
diff_mse = (df_cov_mse - df_wocov_mse) / df_wocov_mse * 100
diff_mae = (df_cov_mae - df_wocov_mae) / df_wocov_mae * 100
diff_avg = (diff_mae + diff_mse) / 2

# print diff_avg for ID and OOD and round to 2 decimals
models = diff_avg.index
for model in models:
    print(model)
    for num in diff_avg.loc[model, ood_cols].round(2):
        # if num > 0, then prepend '\textcolor{red}{+' and append '}' to num
        if num > 0:
            out = r'& \multicolumn{2}{c}{\textcolor{red}{+' + str(num) + '\%}}'
            print(out)
        # if num < 0, then prepend '\textcolor{blue}{' and append '}' to num
        elif num < 0:
            out = r'& \multicolumn{2}{c}{\textcolor{blue}{' + str(num) + '\%}}'
            print(out)

arima
linreg
& \multicolumn{2}{c}{\textcolor{red}{+82.7\%}}
& \multicolumn{2}{c}{\textcolor{red}{+3.8\%}}
& \multicolumn{2}{c}{\textcolor{red}{+930929.99\%}}
& \multicolumn{2}{c}{\textcolor{red}{+66.07\%}}
& \multicolumn{2}{c}{\textcolor{red}{+0.54\%}}
nhits
& \multicolumn{2}{c}{\textcolor{red}{+8.35\%}}
& \multicolumn{2}{c}{\textcolor{red}{+35.49\%}}
& \multicolumn{2}{c}{\textcolor{red}{+110.09\%}}
& \multicolumn{2}{c}{\textcolor{red}{+12.34\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-0.91\%}}
tft
& \multicolumn{2}{c}{\textcolor{red}{+7.91\%}}
& \multicolumn{2}{c}{\textcolor{red}{+4.84\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-1.22\%}}
& \multicolumn{2}{c}{\textcolor{red}{+10.01\%}}
& \multicolumn{2}{c}{\textcolor{red}{+1.41\%}}
transformer
& \multicolumn{2}{c}{\textcolor{blue}{-1.59\%}}
& \multicolumn{2}{c}{\textcolor{red}{+23.61\%}}
& \multicolumn{2}{c}{\textcolor{red}{+117.27\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-10.23\%}}
& \multicolumn{2}{c}{\textcolor{red}{+12.14\%}}
xgboos

In [11]:
diff_mse_id_ood = pd.DataFrame()
diff_mae_id_ood = pd.DataFrame()
pairs = []
for col in diff_mse.columns:
    if col.endswith('ID'):
        pair = (col, col[:-3] + ' OOD')
        pairs.append(pair)
for pair in pairs:
    col_name = pair[0] + '_' + pair[1]
    diff_cov_mse_id_ood = (df_cov_mse[pair[1]] - df_cov_mse[pair[0]]) / df_cov_mse[pair[0]] * 100
    diff_wocov_mse_id_ood = (df_wocov_mse[pair[1]] - df_wocov_mse[pair[0]]) / df_wocov_mse[pair[0]] * 100
    diff_mse_id_ood[col_name] = np.minimum(diff_cov_mse_id_ood, diff_wocov_mse_id_ood)
    diff_cov_mae_id_ood = (df_cov_mae[pair[1]] - df_cov_mae[pair[0]]) / df_cov_mae[pair[0]] * 100
    diff_wocov_mae_id_ood = (df_wocov_mae[pair[1]] - df_wocov_mae[pair[0]]) / df_wocov_mae[pair[0]] * 100
    diff_mae_id_ood[col_name] = np.minimum(diff_cov_mae_id_ood, diff_wocov_mae_id_ood)
for model in models:
    print(model)
    for mse, mae in zip(diff_mse_id_ood.loc[model], 
                        diff_mae_id_ood.loc[model]):
        avg = float((mse + mae) / 2)
        # round avg to 2 decimals
        avg = round(avg, 2)
        if avg > 0:
            out = r'& \multicolumn{2}{c}{\textcolor{red}{+' + str(avg) + '\%}}'
            print(out)
        else:
            out = r'& \multicolumn{2}{c}{\textcolor{blue}{' + str(avg) + '\%}}'
            print(out)

arima
& \multicolumn{2}{c}{\textcolor{blue}{nan\%}}
& \multicolumn{2}{c}{\textcolor{blue}{nan\%}}
& \multicolumn{2}{c}{\textcolor{blue}{nan\%}}
& \multicolumn{2}{c}{\textcolor{blue}{nan\%}}
& \multicolumn{2}{c}{\textcolor{blue}{nan\%}}
linreg
& \multicolumn{2}{c}{\textcolor{red}{+1.9\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-0.45\%}}
& \multicolumn{2}{c}{\textcolor{red}{+24.71\%}}
& \multicolumn{2}{c}{\textcolor{red}{+5.52\%}}
& \multicolumn{2}{c}{\textcolor{red}{+14.36\%}}
nhits
& \multicolumn{2}{c}{\textcolor{blue}{-3.8\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-17.01\%}}
& \multicolumn{2}{c}{\textcolor{red}{+4.86\%}}
& \multicolumn{2}{c}{\textcolor{red}{+0.78\%}}
& \multicolumn{2}{c}{\textcolor{red}{+7.29\%}}
tft
& \multicolumn{2}{c}{\textcolor{blue}{-9.91\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-11.11\%}}
& \multicolumn{2}{c}{\textcolor{blue}{-6.84\%}}
& \multicolumn{2}{c}{\textcolor{red}{+2.41\%}}
& \multicolumn{2}{c}{\textcolor{red}{+3.18\%}}
transformer
& \multicolumn{2}{c}{\

# Clearing the results

In [ ]:
import os

# Set the directory path to the folder containing the output files
folder_path = './output'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    # Open the file in read mode if the file starts with transformer
    if filename.startswith('transformer') or \
            filename.startswith('tft') or \
                filename.startswith('linreg') or \
                    filename.startswith('xgboost') or \
                        filename.startswith('nhits'):
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        # Loop through the lines in reverse order
        for i in range(len(lines)-1, -1, -1):
            if lines[i].startswith('Best value: '):
                # Delete all lines after the line starting with "Best value: "
                del lines[i+1:]
                break
        
        # Open the file in write mode and write the modified lines back to the file
        with open(file_path, 'w') as f:
            f.writelines(lines)
